# Running a Federated Cycle with Synergos

This tutorial aims to give you an understanding of how to use the synergos package to run a full federated learning cycle. 

In a federated learning system, there are many contributory participants, known as Worker nodes, which receive a global model to train on, with their own local dataset. The dataset does not leave the individual Worker nodes at any point, and remains private to the node.

The job to synchronize, orchestrate and initiate an federated learning cycle, falls on a Trusted Third Party (TTP). The TTP pushes out the global model architecture and parameters for the individual nodes to train on, calling upon the required data, based on tags, e.g "training", which points to relevant data on the individual nodes. At no point does the TTP receive, copy or access the Worker nodes' local datasets.

In this tutorial, you will go through the steps required by each participant (TTP and Worker), by simulating each of them locally with docker containers. Specifically, we will simulate a TTP and 2 Workers. 

At the end of this, we will have:
- Connected the participants
- Trained the model
- Evaluate the model

## About the Dataset and Task

The dataset used in this notebook is on a small subset of Federated EMNIST (FEMNIST) images, comprising 3 classes, and all images are 28 x 28 pixels. The dataset is available in the same directory as this notebook. Within the dataset directory, `data1` is for Worker 1 and `data2` is for Worker 2. The task to be carried out will be a multi-classification.

The dataset we have provided is a processed subset of the original FEMNIST dataset retrieved from [here](https://github.com/TalwalkarLab/leaf/tree/master/data/femnist).

## Initiating the docker containers

Before we begin, we have to start the docker containers.

Firstly, pull the required docker images with the following commands:
1. Synergos TTP (Basic):

`docker pull registry.aisingapore.net/fedlearn/synergos_ttp:syn0.7`

2. Synergos Worker:

`docker pull registry.aisingapore.net/fedlearn/synergos_worker:syn0.5`

Next, in <u>separate</u> CLI terminals, run the following command:

**Note: For Windows users, it is advisable to use powershell or command prompt based interfaces**

**Worker 1**

```docker run -v <directory femnist/data1>:/worker/data -v <directory femnist/outputs_1>:/worker/outputs --name worker_1 synergos_worker:v0.1.0 --id worker_1 --logging_variant basic```

**Worker 2**

```docker run -v <directory femnist/data2>:/worker/data -v <directory femnist/outputs_2>:/worker/outputs --name worker_2 synergos_worker:v0.1.0 --id worker_2 --logging_variant basic```

**TTP**

```docker run -p 0.0.0.0:5000:5000 -p 5678:5678 -p 8020:8020 -p 8080:8080 -v <directory femnist/mlflow_test>:/ttp/mlflow -v <directory femnist/ttp_data>:/ttp/data --name ttp --link worker_1 --link worker_2 synergos_ttp:v0.1.0 --id ttp --logging_variant basic -c```


Once ready, for each terminal, you should see that a Flask app is running on http://0.0.0.0:5000 of the container.

You are now ready for the next step.

## Configuration

In a new terminal, run `docker inspect bridge` and find the IPv4Address for each container. Ideally, the containers should have the following addresses:
- worker_1 address: 172.17.0.2
- worker_2 address: 172.17.0.3
- ttp address: 172.17.0.4

If not, just note the relevant IP addresses for each docker container.

Run the following cells below.

**Note: For Windows users, `host` should be Docker Desktop VM's IP. Follow [this](https://stackoverflow.com/questions/58073936/how-to-get-ip-address-of-docker-desktop-vm) on instructions to find IP**

In [1]:
from synergos import Driver

host = "172.20.0.2"
port = 5000

# Initiate Driver
driver = Driver(host=host, port=port)

## Phase 1: Connect

Submitting TTP & Participant metadata

#### 1A. Orchestrator creates a collaboration

In [2]:
collab_task = driver.collaborations

collab_task.configure_logger(
    host="172.20.0.14", 
    port=9000, 
    sysmetrics_port=9100, 
    director_port=9200, 
    ttp_port=9300, 
    worker_port=9400, 
    ui_port=9000, 
    secure=False
)

collab_task.configure_mlops( 
    host="172.20.0.15", 
    port=5500, 
    ui_port=5500, 
    secure=False
)

collab_task.configure_mq( 
    host="172.20.0.16", 
    port=5672, 
    ui_port=15672, 
    secure=False
)

collab_task.create('femnist_syncluster_collaboration')

{'data': {'doc_id': '2',
  'kind': 'Collaboration',
  'key': {'collab_id': 'femnist_syncluster_collaboration'},
  'relations': {'Project': [],
   'Experiment': [],
   'Run': [],
   'Registration': [],
   'Tag': [],
   'Model': [],
   'Validation': [],
   'Prediction': []},
  'logs': {'host': '172.20.0.14',
   'ports': {'sysmetrics': 9100,
    'director': 9200,
    'ttp': 9300,
    'worker': 9400,
    'main': 9000,
    'ui': 9000},
   'secure': False},
  'mlops': {'host': '172.20.0.15',
   'ports': {'main': 5500, 'ui': 5500},
   'secure': False},
  'mq': {'host': '172.20.0.16',
   'ports': {'main': 5672, 'ui': 15672},
   'secure': False}},
 'apiVersion': '0.2.0',
 'success': 1,
 'status': 201,
 'method': 'collaborations.post',
 'params': {}}

#### 1B. Orchestrator creates a project

In [3]:
driver.projects.create(
    collab_id="femnist_syncluster_collaboration",
    project_id="femnist_syncluster_project",
    action="classify",
    incentives={
        'tier_1': [],
        'tier_2': [],
    }
)

{'data': {'doc_id': '2',
  'kind': 'Project',
  'key': {'collab_id': 'femnist_syncluster_collaboration',
   'project_id': 'femnist_syncluster_project'},
  'relations': {'Experiment': [],
   'Run': [],
   'Registration': [],
   'Tag': [],
   'Model': [],
   'Validation': [],
   'Prediction': []},
  'action': 'classify',
  'incentives': {'tier_2': [], 'tier_1': []}},
 'apiVersion': '0.2.0',
 'success': 1,
 'status': 201,
 'method': 'projects.post',
 'params': {'collab_id': 'femnist_syncluster_collaboration'}}

#### 1C. Orchestrator creates an experiment

In [4]:
driver.experiments.create(
    collab_id="femnist_syncluster_collaboration",
    project_id="femnist_syncluster_project",
    expt_id="femnist_syncluster_experiment",
    model=[
        {
            "activation": "relu",
            "is_input": True,
            "l_type": "Conv2d",
            "structure": {
                "in_channels": 1, 
                "out_channels": 4,
                "kernel_size": 3,
                "stride": 1,
                "padding": 1
            }
        },
        {
            "activation": None,
            "is_input": False,
            "l_type": "Flatten",
            "structure": {}
        },
        {
            "activation": "softmax",
            "is_input": False,
            "l_type": "Linear",
            "structure": {
                "bias": True,
                "in_features": 4 * 28 * 28,
                "out_features": 3
            }
        }

    ]
)

{'apiVersion': '0.2.0',
 'success': 1,
 'status': 201,
 'method': 'experiments.post',
 'params': {'collab_id': 'femnist_syncluster_collaboration',
  'project_id': 'femnist_syncluster_project'},
 'data': {'created_at': '2021-09-06 19:35:18 N',
  'key': {'collab_id': 'femnist_syncluster_collaboration',
   'expt_id': 'femnist_syncluster_experiment',
   'project_id': 'femnist_syncluster_project'},
  'model': [{'activation': 'relu',
    'is_input': True,
    'l_type': 'Conv2d',
    'structure': {'in_channels': 1,
     'kernel_size': 3,
     'out_channels': 4,
     'padding': 1,
     'stride': 1}},
   {'activation': None,
    'is_input': False,
    'l_type': 'Flatten',
    'structure': {}},
   {'activation': 'softmax',
    'is_input': False,
    'l_type': 'Linear',
    'structure': {'bias': True, 'in_features': 3136, 'out_features': 3}}],
  'relations': {'Run': [], 'Model': [], 'Validation': [], 'Prediction': []},
  'doc_id': 2,
  'kind': 'Experiment'}}

#### 1D. Orchestrator creates a run

In [5]:
driver.runs.create(
    collab_id="femnist_syncluster_collaboration",
    project_id="femnist_syncluster_project",
    expt_id="femnist_syncluster_experiment",
    run_id="femnist_syncluster_run",
    rounds=2, 
    epochs=1,
    base_lr=0.0005,
    max_lr=0.005,
    criterion="BCELoss"
)

{'data': {'doc_id': '2',
  'kind': 'Run',
  'key': {'collab_id': 'femnist_syncluster_collaboration',
   'project_id': 'femnist_syncluster_project',
   'expt_id': 'femnist_syncluster_experiment',
   'run_id': 'femnist_syncluster_run'},
  'relations': {'Model': [], 'Validation': [], 'Prediction': []},
  'rounds': 2,
  'epochs': 1,
  'lr': 0.001,
  'lr_decay': 0.1,
  'weight_decay': 0.0,
  'seed': 42,
  'precision_fractional': 5,
  'mu': 0.1,
  'l1_lambda': 0.0,
  'l2_lambda': 0.0,
  'base_lr': 0.0005,
  'max_lr': 0.005,
  'patience': 10,
  'delta': 0.0},
 'apiVersion': '0.2.0',
 'success': 1,
 'status': 201,
 'method': 'runs.post',
 'params': {'collab_id': 'femnist_syncluster_collaboration',
  'project_id': 'femnist_syncluster_project',
  'expt_id': 'femnist_syncluster_experiment'}}

#### 1E. Participants registers their servers' configurations and roles

In [6]:
participant_resp_1 = driver.participants.create(
    participant_id="worker_1",
)

display(participant_resp_1)

participant_resp_2 = driver.participants.create(
    participant_id="worker_2",
)

display(participant_resp_2)

{'data': {'doc_id': '1',
  'kind': 'Participant',
  'key': {'participant_id': 'worker_1'},
  'relations': {'Registration': [{'doc_id': '1',
     'kind': 'Registration',
     'key': {'collab_id': 'femnist_synplus_collaboration',
      'project_id': 'femnist_synplus_project',
      'participant_id': 'worker_1'},
     'collaboration': {'catalogue': {},
      'logs': {},
      'meter': {},
      'mlops': {},
      'mq': {}},
     'project': {'action': None, 'incentives': {}, 'start_at': None},
     'participant': {'id': None,
      'category': [],
      'summary': None,
      'phone': None,
      'email': None,
      'socials': {}},
     'role': 'host',
     'n_count': 1,
     'node_0': {'host': '172.19.0.3',
      'port': 8020,
      'log_msgs': True,
      'verbose': True,
      'f_port': 5000}}],
   'Tag': [{'doc_id': '1',
     'kind': 'Tag',
     'key': {'collab_id': 'femnist_synplus_collaboration',
      'project_id': 'femnist_synplus_project',
      'participant_id': 'worker_1'},
   

{'data': {'doc_id': '2',
  'kind': 'Participant',
  'key': {'participant_id': 'worker_2'},
  'relations': {'Registration': [{'doc_id': '2',
     'kind': 'Registration',
     'key': {'collab_id': 'femnist_synplus_collaboration',
      'project_id': 'femnist_synplus_project',
      'participant_id': 'worker_2'},
     'collaboration': {'catalogue': {},
      'logs': {},
      'meter': {},
      'mlops': {},
      'mq': {}},
     'project': {'action': None, 'incentives': {}, 'start_at': None},
     'participant': {'id': None,
      'category': [],
      'summary': None,
      'phone': None,
      'email': None,
      'socials': {}},
     'role': 'guest',
     'n_count': 1,
     'node_0': {'host': '172.19.0.4',
      'port': 8020,
      'log_msgs': True,
      'verbose': True,
      'f_port': 5000}}],
   'Tag': [{'doc_id': '2',
     'kind': 'Tag',
     'key': {'collab_id': 'femnist_synplus_collaboration',
      'project_id': 'femnist_synplus_project',
      'participant_id': 'worker_2'},
  

In [7]:
registration_task = driver.registrations

# Add and register worker_1 node
registration_task.add_node(
    host='172.20.0.4',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

registration_task.add_node(
    host='172.20.0.7',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

registration_task.create(
    collab_id="femnist_syncluster_collaboration",
    project_id="femnist_syncluster_project",
    participant_id="worker_1",
    role="host"
)

{'data': {'doc_id': '3',
  'kind': 'Registration',
  'key': {'collab_id': 'femnist_syncluster_collaboration',
   'project_id': 'femnist_syncluster_project',
   'participant_id': 'worker_1'},
  'relations': {'Tag': []},
  'collaboration': {'catalogue': {},
   'logs': {'host': '172.20.0.14',
    'ports': {'sysmetrics': 9100,
     'director': 9200,
     'ttp': 9300,
     'worker': 9400,
     'main': 9000,
     'ui': 9000},
    'secure': False},
   'meter': {},
   'mlops': {'host': '172.20.0.15',
    'ports': {'main': 5500, 'ui': 5500},
    'secure': False},
   'mq': {'host': '172.20.0.16',
    'ports': {'main': 5672, 'ui': 15672},
    'secure': False}},
  'project': {'action': 'classify',
   'incentives': {'tier_2': [], 'tier_1': []},
   'start_at': None},
  'participant': {'id': 'worker_1',
   'category': [],
   'summary': None,
   'phone': None,
   'email': None,
   'socials': {}},
  'role': 'host',
  'n_count': 2,
  'node_1': {'host': '172.20.0.7',
   'port': 8020,
   'log_msgs': True,

In [8]:
registration_task = driver.registrations

# Add and register worker_2 node
registration_task.add_node(
    host='172.20.0.5',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

registration_task.add_node(
    host='172.20.0.8',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

registration_task.create(
    collab_id="femnist_syncluster_collaboration",
    project_id="femnist_syncluster_project",
    participant_id="worker_2",
    role="guest"
)

{'data': {'doc_id': '4',
  'kind': 'Registration',
  'key': {'collab_id': 'femnist_syncluster_collaboration',
   'project_id': 'femnist_syncluster_project',
   'participant_id': 'worker_2'},
  'relations': {'Tag': []},
  'collaboration': {'catalogue': {},
   'logs': {'host': '172.20.0.14',
    'ports': {'sysmetrics': 9100,
     'director': 9200,
     'ttp': 9300,
     'worker': 9400,
     'main': 9000,
     'ui': 9000},
    'secure': False},
   'meter': {},
   'mlops': {'host': '172.20.0.15',
    'ports': {'main': 5500, 'ui': 5500},
    'secure': False},
   'mq': {'host': '172.20.0.16',
    'ports': {'main': 5672, 'ui': 15672},
    'secure': False}},
  'project': {'action': 'classify',
   'incentives': {'tier_2': [], 'tier_1': []},
   'start_at': None},
  'participant': {'id': 'worker_2',
   'category': [],
   'summary': None,
   'phone': None,
   'email': None,
   'socials': {}},
  'role': 'guest',
  'n_count': 2,
  'node_1': {'host': '172.20.0.8',
   'port': 8020,
   'log_msgs': True

#### 1F. Participants registers their tags for a specific project

In [9]:
# Worker 1 declares their data tags
driver.tags.create(
    collab_id="femnist_syncluster_collaboration",
    project_id="femnist_syncluster_project",
    participant_id="worker_1",
    train=[["femnist", "dataset", "data1", "train"]],
    evaluate=[["femnist", "dataset", "data1", "evaluate"]]
)

{'data': {'doc_id': '3',
  'kind': 'Tag',
  'key': {'collab_id': 'femnist_syncluster_collaboration',
   'project_id': 'femnist_syncluster_project',
   'participant_id': 'worker_1'},
  'train': [['femnist', 'dataset', 'data1', 'train']],
  'evaluate': [['femnist', 'dataset', 'data1', 'evaluate']],
  'predict': []},
 'apiVersion': '0.2.0',
 'success': 1,
 'status': 201,
 'method': 'tag.post',
 'params': {'collab_id': 'femnist_syncluster_collaboration',
  'project_id': 'femnist_syncluster_project',
  'participant_id': 'worker_1'}}

In [10]:
# Worker 2 declares their data tags
driver.tags.create(
    collab_id="femnist_syncluster_collaboration",
    project_id="femnist_syncluster_project",
    participant_id="worker_2",
    train=[["femnist", "dataset", "data2", "train"]],
    evaluate=[["femnist", "dataset", "data2", "evaluate"]]
)

{'data': {'doc_id': '4',
  'kind': 'Tag',
  'key': {'collab_id': 'femnist_syncluster_collaboration',
   'project_id': 'femnist_syncluster_project',
   'participant_id': 'worker_2'},
  'train': [['femnist', 'dataset', 'data2', 'train']],
  'evaluate': [['femnist', 'dataset', 'data2', 'evaluate']],
  'predict': []},
 'apiVersion': '0.2.0',
 'success': 1,
 'status': 201,
 'method': 'tag.post',
 'params': {'collab_id': 'femnist_syncluster_collaboration',
  'project_id': 'femnist_syncluster_project',
  'participant_id': 'worker_2'}}

## Phase 2: 
Alignment, Training & Optimisation

#### 2A. Perform multiple feature alignment to dynamically configure datasets and models for cross-grid compatibility

In [11]:
driver.alignments.create(
    collab_id='femnist_syncluster_collaboration',
    project_id="femnist_syncluster_project",
    verbose=False,
    log_msg=False
)

{'data': [],
 'apiVersion': '0.2.0',
 'success': 1,
 'status': 201,
 'method': 'alignments.post',
 'params': {'collab_id': 'femnist_syncluster_collaboration',
  'project_id': 'femnist_syncluster_project'}}

In [12]:
# Run this cell again after alignment has completed to retrieve your alignments
driver.alignments.read(
    collab_id='femnist_syncluster_collaboration',
    project_id="femnist_syncluster_project"
)

{'message': "MFA has not been performed for Project 'femnist_syncluster_project'!"}

#### 2B. Trigger training across the federated grid

In [13]:
model_resp = driver.models.create(
    collab_id="femnist_syncluster_collaboration",
    project_id="femnist_syncluster_project",
    expt_id="femnist_syncluster_experiment",
    run_id="femnist_syncluster_run",
    log_msg=False,
    verbose=False
)
display(model_resp)

{'data': [],
 'apiVersion': '0.2.0',
 'success': 1,
 'status': 200,
 'method': 'models.post',
 'params': {'collab_id': 'femnist_syncluster_collaboration',
  'project_id': 'femnist_syncluster_project',
  'expt_id': 'femnist_syncluster_experiment',
  'run_id': 'femnist_syncluster_run'}}

In [14]:
# Run this cell again after training has completed to retrieve your model metadata
driver.models.read(
    collab_id="femnist_syncluster_collaboration",
    project_id="femnist_syncluster_project",
    expt_id="femnist_syncluster_experiment",
    run_id="femnist_syncluster_run"
)

{'message': 'Models does not exist for specified keyword filters!'}

#### 2C. Perform hyperparameter tuning once ideal model is found (optional) 

## Phase 3: EVALUATE 
Validation & Predictions

#### 3A. Perform validation(s) of combination(s)

In [15]:
# Orchestrator performs post-mortem validation
driver.validations.create(
    collab_id="femnist_syncluster_collaboration",
    project_id="femnist_syncluster_project",
    expt_id="femnist_syncluster_experiment",
    run_id="femnist_syncluster_run",
    log_msg=False,
    verbose=False
)

{'data': [],
 'apiVersion': '0.2.0',
 'success': 1,
 'status': 200,
 'method': 'validations.post',
 'params': {'collab_id': 'femnist_syncluster_collaboration',
  'project_id': 'femnist_syncluster_project',
  'expt_id': 'femnist_syncluster_experiment',
  'run_id': 'femnist_syncluster_run'}}

In [16]:
# Run this cell again after validation has completed to retrieve your validation statistics
driver.validations.read(
    collab_id="femnist_syncluster_collaboration",
    project_id="femnist_syncluster_project",
    expt_id="femnist_syncluster_experiment",
    run_id="femnist_syncluster_run"
)

{'message': 'Validations do not exist for specified keyword filters!'}

#### 3B. Perform prediction(s) of combination(s)

In [17]:
# Worker 1 requests for inferences
driver.predictions.create(
    tags={"femnist_syncluster_project": [["femnist", "dataset", "data1", "predict"]]},
    participant_id="worker_1",
    collab_id="femnist_syncluster_collaboration",
    project_id="femnist_syncluster_project",
    expt_id="femnist_syncluster_experiment",
    run_id="femnist_syncluster_run",
    log_msg=False,
    verbose=False
)

{'data': [],
 'apiVersion': '0.2.0',
 'success': 1,
 'status': 200,
 'method': 'predictions.post',
 'params': {'collab_id': 'femnist_syncluster_collaboration',
  'project_id': 'femnist_syncluster_project',
  'expt_id': 'femnist_syncluster_experiment',
  'run_id': 'femnist_syncluster_run',
  'participant_id': 'worker_1'}}

In [18]:
# Run this cell again after prediction has completed to retrieve your predictions for worker 1
driver.predictions.read(
    participant_id="worker_1",
    collab_id="femnist_syncluster_collaboration",
    project_id="femnist_syncluster_project",
    expt_id="femnist_syncluster_experiment",
    run_id="femnist_syncluster_run"
)

{'message': 'Predictions do not exist for specified keyword filters!'}

In [19]:
# Worker 2 requests for inferences
driver.predictions.create(
    tags={"femnist_syncluster_project": [["femnist", "dataset", "data2", "predict"]]},
    participant_id="worker_2",
    collab_id="femnist_syncluster_collaboration",
    project_id="femnist_syncluster_project",
    expt_id="femnist_syncluster_experiment",
    run_id="femnist_syncluster_run",
    log_msg=False,
    verbose=False
)

{'data': [],
 'apiVersion': '0.2.0',
 'success': 1,
 'status': 200,
 'method': 'predictions.post',
 'params': {'collab_id': 'femnist_syncluster_collaboration',
  'project_id': 'femnist_syncluster_project',
  'expt_id': 'femnist_syncluster_experiment',
  'run_id': 'femnist_syncluster_run',
  'participant_id': 'worker_2'}}

In [20]:
# Run this cell again after prediction has completed to retrieve your predictions for worker 2
driver.predictions.read(
    participant_id="worker_2",
    collab_id="femnist_syncluster_collaboration",
    project_id="femnist_syncluster_project",
    expt_id="femnist_syncluster_experiment",
    run_id="femnist_syncluster_run"
)

{'message': 'Predictions do not exist for specified keyword filters!'}